<a href="https://colab.research.google.com/github/hxf829/paddle/blob/main/01_paddle_housing_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
! cat /usr/local/cuda/version.txt

CUDA Version 10.1.243


In [ ]:
!python -m pip install paddlepaddle-gpu==1.8.5.post107 -i https://mirror.baidu.com/pypi/simple

In [5]:
import paddle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import paddle.fluid as fluid
import paddle.fluid.dygraph as dygraph
from paddle.fluid.dygraph import Linear
import numpy as np
import os
import random

In [6]:
def load_data():
    train_data = np.asarray(pd.read_csv('./sample_data/california_housing_train.csv'))
    test_data = np.asarray(pd.read_csv('./sample_data/california_housing_test.csv'))
    maxmums, minmums, means = train_data.max(axis=0), train_data.min(axis=0), train_data.mean(axis=0)
    for i in range(train_data[0].shape[0]):
        train_data[:, i] = (train_data[:, i] - minmums[i]) / (maxmums[i] - minmums[i] + 1e-05) 
    for i in range(test_data[0].shape[0]):
        test_data[:, i] = (test_data[:, i] - minmums[i]) / (maxmums[i] - minmums[i] + 1e-05) 
    return train_data, test_data

In [7]:
class Regressor(fluid.dygraph.Layer):
    def __init__(self):
        super(Regressor, self).__init__()
        
        # 定义一层全连接层，输出维度是1，激活函数为None，即不使用激活函数
        self.fc = Linear(input_dim=8, output_dim=1, act=None)
    
    # 网络的前向计算函数
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

In [8]:
# 定义飞桨动态图的工作环境
#with fluid.dygraph.guard():
    

In [14]:
with dygraph.guard(fluid.CUDAPlace(0)):
    # 声明定义好的线性回归模型
    model = Regressor()
    # 开启模型训练模式
    model.train()
    # 加载数据
    training_data, test_data = load_data()
    # 定义优化算法，这里使用随机梯度下降-SGD
    # 学习率设置为0.01
    opt = fluid.optimizer.SGD(learning_rate=0.01, parameter_list=model.parameters())
    EPOCH_NUM = 10   # 设置外层循环次数
    BATCH_SIZE = 10  # 设置batch大小
    
    # 定义外层循环
    for epoch_id in range(EPOCH_NUM):
        # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
        np.random.shuffle(training_data)
        # 将训练数据进行拆分，每个batch包含10条数据
        mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
        # 定义内层循环
        for iter_id, mini_batch in enumerate(mini_batches):
            x = np.array(mini_batch[:, :-1]).astype('float32') # 获得当前批次训练数据
            y = np.array(mini_batch[:, -1:]).astype('float32') # 获得当前批次训练标签（真实房价）
            # 将numpy数据转为飞桨动态图variable形式
            house_features = dygraph.to_variable(x)
            prices = dygraph.to_variable(y)
            
            # 前向计算
            predicts = model(house_features)
            
            # 计算损失
            loss = fluid.layers.square_error_cost(predicts, label=prices)
            avg_loss = fluid.layers.mean(loss)
            if iter_id%20==0:
                print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
            
            # 反向传播
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.minimize(avg_loss)
            # 清除梯度
            model.clear_gradients()
    # 保存模型
    fluid.save_dygraph(model.state_dict(), 'LR_model')

epoch: 0, iter: 0, loss is: [0.18165639]
epoch: 0, iter: 20, loss is: [0.09504198]
epoch: 0, iter: 40, loss is: [0.07041156]
epoch: 0, iter: 60, loss is: [0.1367161]
epoch: 0, iter: 80, loss is: [0.11435135]
epoch: 0, iter: 100, loss is: [0.24962457]
epoch: 0, iter: 120, loss is: [0.13988951]
epoch: 0, iter: 140, loss is: [0.04382548]
epoch: 0, iter: 160, loss is: [0.04360772]
epoch: 0, iter: 180, loss is: [0.18054366]
epoch: 0, iter: 200, loss is: [0.07448409]
epoch: 0, iter: 220, loss is: [0.08119582]
epoch: 0, iter: 240, loss is: [0.07348886]
epoch: 0, iter: 260, loss is: [0.14950344]
epoch: 0, iter: 280, loss is: [0.09649649]
epoch: 0, iter: 300, loss is: [0.04066939]
epoch: 0, iter: 320, loss is: [0.12653047]
epoch: 0, iter: 340, loss is: [0.12966368]
epoch: 0, iter: 360, loss is: [0.08446094]
epoch: 0, iter: 380, loss is: [0.12081803]
epoch: 0, iter: 400, loss is: [0.06892101]
epoch: 0, iter: 420, loss is: [0.0385909]
epoch: 0, iter: 440, loss is: [0.0542938]
epoch: 0, iter: 460,